In [16]:
# Import general libraries
import json
import pandas as pd

# Import local functions
from general_functions import awsOps

In [17]:
legal_prompt = ("""
I need you to summarize any important legal information in a news article.
Only write the relevant bullet points, no headline, no greeting etc.
State any relevant developments from law and legislation.

I will give you one example of how you should respond:
Example News Article from me:
The Women's Retirement Protection Act of 2023 is a proposed legislation that aims to establish various spousal protections for 401(k) retirement accounts. Among the provisions outlined in the act are measures to prevent spousal discrimination in retirement savings plans, requirements for spousal consent for certain distributions, and enhanced survivor benefits for nonworking spouses. This act aims to address the long-standing gender disparities in retirement savings and ensure that women are adequately protected and provided for in their retirement years.
Example Answer from you:
1) The Women's Retirement Protection Act of 2023 proposes spousal protections for 401(k) retirement accounts.
2) Provisions include measures to prevent spousal discrimination in retirement savings plans.
3) The act requires spousal consent for certain distributions and enhances survivor benefits for nonworking spouses.
4) The aim is to address gender disparities in retirement savings and enhance retirement security for women.
             
Real News Article from me:   
{article}
Real Answer from you:
""")

In [18]:
sanctions_prompt = ("""
I need you to summarize any information on sanctions in a news article.
Only write the relevant bullet points, no headline, no greeting etc.
State any relevant developments from trials and fines.

I will give you one example of how you should respond:
Example News Article from me:
The former CEO of M.M. Warburg bank, Kristians Oleariuss, appeared in a court in the German city of Bonn last month. This is just one of the cases related to the Cum-Ex tax scandal that has rocked Germany and several other European countries. Oleariuss and other individuals are accused of participating in a scheme that allowed multiple people to claim tax rebates on the same dividend. The scandal is estimated to have cost European countries billions of euros in lost tax revenue. The trial serves as a reminder of the ongoing legal battle surrounding the Cum-Ex scandal.
Example Answer from you:
1. Former CEO of M.M. Warburg bank, Kristians Oleariuss, appeared in court in Bonn, Germany, related to the Cum-Ex tax scandal.
2. The Cum-Ex scandal involves a scheme where multiple individuals claimed tax rebates on the same dividend, impacting Germany and other European countries.
3. The scandal is estimated to have caused billions of euros in lost tax revenue for European countries.
4. The trial underscores the continuing legal repercussions and the broader legal battle concerning the Cum-Ex scandal.
             
Real News Article from me:   
{article}
Real Answer from you:
""")

In [19]:
papers_prompt = ("""
I need you to summarize any information on papers in a news article.
Only write the relevant bullet points, no headline, no greeting etc.
State any relevant developments scientific research.

I will give you one example of how you should respond:
Example News Article from me:
UBS has released a white paper titled 'The future of finance: why banks must embrace open banking,' which discusses how open banking will change the traditional banking model. The paper highlights that banks should view open banking not as a threat, rather as a way to create more personalized customer experiences and increase revenue. The report also discusses the potential for banks to collaborate with fintechs to drive innovation, with the partnership being a win-win situation for both parties. However, the paper warns that the adoption of open banking must be done carefully to protect customer data and ensure regulatory compliance.
Example Answer from you:
1. UBS's white paper, "The future of finance: why banks must embrace open banking," discusses the transformation of traditional banking through open banking.
2. The paper suggests banks should perceive open banking as an opportunity to enhance personalized customer experiences and increase revenue.
3. It advocates for collaboration between banks and fintechs to foster innovation, benefiting both sectors.
4. The paper emphasizes the need for cautious adoption of open banking to safeguard customer data and adhere to regulatory compliance.
             
Real News Article from me:   
{article}
Real Answer from you:
""")

In [20]:
reports_prompt = ("""
I need you to summarize any information on reports in a news article.
Only write the relevant bullet points, no headline, no greeting etc.
State any relevant developments on reports or surveys mentioned.

I will give you one example of how you should respond:
Example News Article from me:
Julius Baer, a Swiss private bank, has been named the best bank in the world for High Net Worth Individuals (HNWIs) by Euromoney magazine. Euromoney's annual Private Banking and Wealth Management Survey recognized Julius Baer for its exceptional services and expertise in catering to the needs of HNWIs. The bank's strong commitment to client satisfaction and its ability to offer tailored investment strategies played a significant role in securing the top spot. Julius Baer's focus on providing a comprehensive range of wealth management solutions and its dedication to maintaining the highest standards of confidentiality were also highlighted as key factors in its success. With this accolade, Julius Baer solidifies its position as a leading private bank and reinforces its reputation for delivering exceptional financial services to the world's wealthiest individuals.
Example Answer from you:
1) Julius Baer has been awarded the title of the best bank for High Net Worth Individuals (HNWIs) by Euromoney magazine.
2) The accolade was given as part of Euromoney's annual Private Banking and Wealth Management Survey, recognizing Julius Baer's superior services and expertise for HNWIs.
3) The bank's commitment to client satisfaction and ability to provide customized investment strategies were instrumental in achieving this recognition.
4) Julius Baer's emphasis on offering a wide array of wealth management solutions and upholding the highest confidentiality standards contributed to its success.
5) This honor reinforces Julius Baer's status as a leading private bank and affirms its excellence in serving the financial needs of the world's wealthiest individuals.
             
Real News Article from me:   
{article}
Real Answer from you:
""")

In [21]:
statement_prompt = ("""
I need you to summarize any important statements in a news article.
Only write the relevant bullet points, no headline, no greeting etc.
State any relevant developments and official statements.

I will give you one example of how you should respond:
Example News Article from me:
Edward Jones, a financial services firm, has announced its plans to launch checking and savings accounts for its U.S. clients by 2024. The move comes as the company aims to further expand its offerings and enhance its client engagement. This strategic decision will allow Edward Jones to provide a more comprehensive range of services to its customers, in addition to its existing offerings in investment and retirement planning. The firm's foray into the checking and savings account space is expected to attract new clients and strengthen its position in the financial services market.
Example Answer from you:
1) Edward Jones plans to launch checking and savings accounts for U.S. clients by 2024.
2) The initiative is part of the company's strategy to expand its offerings and enhance client engagement.
3) This expansion allows Edward Jones to offer a broader range of services, complementing its existing investment and retirement planning services.
4) The firm anticipates that entering the checking and savings account sector will attract new clients and reinforce its market position in financial services.
             
Real News Article from me:   
{article}
Real Answer from you:
""")

In [22]:
guidelines_prompt = ("""
I need you to summarize any information on guidelines in a news article.
Only write the relevant bullet points, no headline, no greeting etc.
State any relevant developments around released guidelines or instructions.

I will give you one example of how you should respond:
Example News Article from me:
Google has advised its employees not to input confidential information into internal chatbots that use artificial intelligence as a result of ongoing privacy concerns. This follows news that Facebook had closed down an AI experiment after two of its chatbots created a language that only they understood. Google\u2019s DeepMind AI recently created a system that enabled bots to communicate with each other without supervision, although the neural networks were restricted to pre-set phrases and sentences.
Example Answer from you:
1) Google has instructed employees to refrain from entering confidential information into AI-powered internal chatbots due to privacy concerns.
2) This advisory follows an incident where Facebook shut down an AI experiment after the chatbots developed a language only they could understand.
3) Google's DeepMind AI developed a system allowing bots to communicate unsupervised, but their interactions were limited to predefined phrases and sentences.
             
Real News Article from me:   
{article}
Real Answer from you:
""")

In [23]:
press_prompt = ("""
I need you to summarize any information on press information in a news article.
Only write the relevant bullet points, no headline, no greeting etc.
State any relevant developments from the press release.

I will give you one example of how you should respond:
Example News Article from me:
German fintech companies are hoping to overcome the challenges presented by low interest rates through a range of strategies, including partnerships with banks and diversification into new products and services. Startups are also shifting their focus to B2B services, offering lending platforms and payment systems to established firms and gaining access to new markets. Some firms are choosing to expand their business by moving into additional markets such as the UK or by introducing new financial products. However, others are struggling to adapt to the changing landscape, with some reporting significant losses.
Example Answer from you:
1) German fintech companies are employing various strategies to counteract the impact of low interest rates.
2) These strategies include forming partnerships with banks and diversifying into novel products and services.
3) Fintech startups are increasingly focusing on B2B services, providing lending platforms and payment systems to established companies to penetrate new markets.
4) Some firms are expanding by entering additional markets like the UK or launching new financial products.
5) Despite these efforts, certain fintech companies are finding it challenging to adjust to the evolving market conditions, with some reporting considerable losses.
             
Real News Article from me:   
{article}
Real Answer from you:
""")

In [24]:
personnel_prompt = ("""
I need you to summarize any important personnel changes in a news article.
Only write bullet points.
Only list events in which an employee was hired or fired.
If possible, state the name of the employee or the number of employees affected and the corresponding positions in the company.

I will give you one example:
Example News Article:
Deutsche Bank has hired a team of Credit Suisse bankers for its Middle East operations in an effort to expand its presence in the region. The move comes as the German lender looks to tap into the potential growth opportunities in the Middle East due to the region's oil wealth. Deutsche Bank has been focusing on strengthening its presence in the Gulf Cooperation Council (GCC) countries, which include Bahrain, Kuwait, Oman, Qatar, Saudi Arabia, and the United Arab Emirates. This move also aligns with the bank's broader strategy of targeting expansion in emerging markets. By attracting top talent from Credit Suisse, Deutsche Bank aims to bolster its ability to serve wealthy individuals and businesses in the region.   
Example Answer:
1. Deutsche Bank has hired a team of Credit Suisse bankers for its Middle East operations.
2. The move involves hiring top talent from Credit Suisse, including an unspecified number of bankers.
3. The employees affected are expected to be responsible for serving wealthy individuals and businesses in the region.
             
Real News Article:   
{article}
Real Answer:             
""")

In [25]:
market_prompt = ("""
I need you to summarize any information on markets in a news article.
Only write the relevant bullet points, no headline, no greeting etc.
State any relevant developments around the financial markets.

I will give you one example of how you should respond:
Example News Article from me:
The article reports that the BVB (Borussia Dortmund) stock has weakened slightly in comparison to previous months. The article highlights that the departing midfielder, Jude Bellingham, had expressed his appreciation for the club and stated that it had been the biggest part of his life.
Example Answer from you:
1) The BVB (Borussia Dortmund) stock has experienced a slight decline compared to previous months.
2) The report includes statements from departing midfielder Jude Bellingham, who expressed gratitude towards the club and noted its significant role in his life.
             
Real News Article from me:   
{article}
Real Answer from you:
""")

In [26]:
multi_prompt = ("""
I need you to summarize any important information in a news article.
Only write the relevant bullet points, no headline, no greeting etc.
State any relevant numbers and developments from the article.
Group your answer into the two sections {cat1} and {cat2}.

I will give you one example of how you should respond:
Example News Article from me:
The Arab Republic of Egypt has announced the successful closure of a $500 million dual-tranche international commercial loan. This loan was equally provided by Deutsche Bank AG and HSBC Bank Middle East Limited. The loan has a term of 7 years and has been arranged by the Ministry of Finance of Egypt. This financing will aid Egypt in meeting its financial requirements and supporting its economic growth. The successful closure of this loan demonstrates international confidence in Egypt's economic prospects.
Example Answer from you:
1. Eqypt has announced the successful closure of a $500 million dual-tranche international commercial loan.
2. The loan was equally provided by Deutsche Bank and HSBC.
3. The loan has a term of 7 years.
4. The loan aids Egypt in meeting its financial requirements and supporting its economic growth.
             
Real News Article from me:   
{article}
Real Answer from you:""")

In [27]:
prompt_dict = {"legal": [legal_prompt],
               "sanctions": [sanctions_prompt],
               "papers": [papers_prompt],
               "reports": [reports_prompt],
               "statements": [statement_prompt],
               "guidelines": [guidelines_prompt],
               "press": [press_prompt],
               "personnel": [personnel_prompt],
               "market": [market_prompt],
               "multi": [multi_prompt]}

prompt_df = pd.DataFrame(prompt_dict)
prompt_df.to_csv("../../data/llama_prompts_v1.csv")

In [28]:
# Upload the data to aws
# Access aws credentials from json file
with open("../aws_credentials.json", 'r') as file:
    aws_creds_json = json.load(file)
# Specify s3 bucket
bucket = "fs-reghub-news-analysis"

# Connect to aws and upload the files
aws = awsOps(aws_creds_json)
aws.upload_file("../../data/llama_prompts_v1.csv", bucket, "llama_prompts_v1.csv")

'File uploaded'